# Linear Programming with PuLP Python Library

### Example 1

You’ll now solve this system with PuLP:

<p align="center">
  <img src="https://files.realpython.com/media/lp-py-eq-2.2984ea2b89df.png" />
</p>

Import the required tools:

In [1]:
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

The first step is to initialize an instance of LpProblem to represent your model:

You use the sense parameter to choose whether to perform minimization (LpMinimize or 1, which is the default) or maximization (LpMaximize or -1). This choice will affect the result of your problem.

In [2]:
model = LpProblem(name="example1", sense=LpMaximize)

Once that you have the model, you can define the decision variables as instances of the LpVariable class:

In [3]:
# You need to provide a lower bound with lowBound=0 because the default value is 
# negative infinity.
x = LpVariable(name="x", lowBound=0)
y = LpVariable(name="y", lowBound=0)

# The parameter upBound defines the upper bound, but you can omit it here because 
# it defaults to positive infinity.

# The optional parameter cat defines the category of a decision variable. If you’re 
# working with continuous variables, then you can use the default value "Continuous".

You can use the variables x and y to create other PuLP objects that represent linear expressions:

In [4]:
# PuLP classes implements Python special methods __add__(), __sub__(), and 
# __mul__() to acheive this.
expression = 2 * x + 4 * y
type(expression)

pulp.pulp.LpAffineExpression

Similarly, you can combine linear expressions, variables, and scalars with the operators ==, <=, or >= to get instances of pulp.LpConstraint that represent the linear constraints of your model.

In [5]:
# PuLP classes implements Python special methods .__eq__(), .__le__(), and
# .__ge__() that define the behavior of the operators ==, <=, and >=.
constraint = 2 * x + 4 * y >= 8
type(constraint)

pulp.pulp.LpConstraint

Add the constraints to the model

In [6]:
# You can append a constraint or objective to the model with the operator 
# += because its class, LpProblem, implements the special method .__iadd__(), 
# which is used to specify the behavior of +=.
model += (2 * x + y <= 20, "red_constraint")
model += (4 * x - 5 * y >= -10, "blue_constraint")
model += (-x + 2 * y >= -2, "yellow_constraint")
model += (-x + 5 * y == 15, "green_constraint")

Add the objective function to the model

In [7]:
model += x + 2 * y

It’s often more convenient to use lpSum():

In [8]:
# It produces the same result as the previous statement.
# model += lpSum([x, 2 * y])
lpSum([x, 2 * y])

1*x + 2*y + 0

You can now see the full definition of this model:

In [9]:
model

example1:
MAXIMIZE
1*x + 2*y + 0
SUBJECT TO
red_constraint: 2 x + y <= 20

blue_constraint: 4 x - 5 y >= -10

yellow_constraint: - x + 2 y >= -2

green_constraint: - x + 5 y = 15

VARIABLES
x Continuous
y Continuous

Finally, you’re ready to solve the problem. You can do that by calling .solve() on your model object. If you want to use the default solver (CBC), then you don’t need to pass any arguments:

In [10]:
# returns the integer status of the solution, which will be 1 if the optimum is found
status = model.solve()
status

1

Here, are the results after optimization:

In [14]:
print(f"Status: {status}, {LpStatus[model.status]}")

Status: 1, Optimal


In [15]:
print(f"Objective: {model.objective.value()}")

Objective: 16.8181817


In [16]:
for var in model.variables():
  print(f"{var.name}: {var.value()}")

x: 7.7272727
y: 4.5454545


In [17]:
for name, constraint in model.constraints.items():
  print(f"{name}: {constraint.value()}")

red_constraint: -9.99999993922529e-08
blue_constraint: 18.181818300000003
yellow_constraint: 3.3636362999999996
green_constraint: -2.0000000233721948e-07


You can see which solver was used by calling .solver:

In [18]:
# You didn’t specify a solver, so PuLP called the default one.
model.solver

You can also use PuLP to solve mixed-integer linear programming problems. To define an integer or binary variable, just pass cat="Integer" or cat="Binary" to LpVariable. Everything else remains the same:

In [19]:
# Create the model
model = LpProblem(name="small-problem", sense=LpMaximize)

# Initialize the decision variables: x is integer, y is continuous
x = LpVariable(name="x", lowBound=0, cat="Integer")
y = LpVariable(name="y", lowBound=0)

# Add the constraints to the model
model += (2 * x + y <= 20, "red_constraint")
model += (4 * x - 5 * y >= -10, "blue_constraint")
model += (-x + 2 * y >= -2, "yellow_constraint")
model += (-x + 5 * y == 15, "green_constraint")

# Add the objective function to the model
model += lpSum([x, 2 * y])

# Solve the problem
status = model.solve()

print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for var in model.variables():
  print(f"{var.name}: {var.value()}")
for name, constraint in model.constraints.items():
  print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 15.8
x: 7.0
y: 4.4
red_constraint: -1.5999999999999996
blue_constraint: 16.0
yellow_constraint: 3.8000000000000007
green_constraint: 0.0


As you can see, the optimal solution is the rightmost green point on the gray background. This is the feasible solution with the largest values of both x and y, giving it the maximal objective function value.

<p align="center">
  <img src="https://files.realpython.com/media/lp-py-fig-6.a415a074213b.png" width=400 />
</p>

### Example 2

Now, we will solve a resource allocation problem. (See reference 1 in [README](./README.md) for details.)

<p align="center">
  <img src="https://files.realpython.com/media/lp-py-eq-4.0178c4cfe357.png" width=400 />
</p>

The approach for defining and solving the problem is the same as in the previous example:

In [20]:
# Define the model
model = LpProblem(name="resource-allocation", sense=LpMaximize)

# Define the decision variables x1, x2, x3 and x4
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 5)}

# Add constraints
model += (lpSum(x.values()) <= 50, "manpower")
model += (3 * x[1] + 2 * x[2] + x[3] <= 100, "material_a")
model += (x[2] + 2 * x[3] + 3 * x[4] <= 90, "material_b")

# Set the objective
model += 20 * x[1] + 12 * x[2] + 40 * x[3] + 25 * x[4]

# Solve the optimization problem
status = model.solve()

# Get the results
print(f"Status: {model.status}, {LpStatus[model.status]}")
print(f"Objective: {model.objective.value()}")

for var in x.values():
  print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
  print(f"{name}: {constraint.value()}")

Status: 1, Optimal
Objective: 1900.0
x1: 5.0
x2: 0.0
x3: 45.0
x4: 0.0
manpower: 0.0
material_a: -40.0
material_b: 0.0


The most profitable solution is to produce 5.0 units of the first product and 45.0 units of the third product per day.

Let’s make this problem more complicated and interesting. Say the factory can’t produce the first and third products in parallel due to a machinery issue. What’s the most profitable solution in this case?

In [21]:
model = LpProblem(name="resource-allocation-2", sense=LpMaximize)

# Define the decision variables
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 5)}
y = {i: LpVariable(name=f"y{i}", cat="Binary") for i in (1, 3)}

# Add constraints
model += (lpSum(x.values()) <= 50, "manpower")
model += (3 * x[1] + 2 * x[2] + x[3] <= 100, "material_a")
model += (x[2] + 2 * x[3] + 3 * x[4] <= 90, "material_b")

M = 100
model += (x[1] <= y[1] * M, "x1_constraint")
model += (x[3] <= y[3] * M, "x3_constraint")
model += (y[1] + y[3] <= 1, "y_constraint")

# Set objective
model += 20 * x[1] + 12 * x[2] + 40 * x[3] + 25 * x[4]

# Solve the optimization problem
status = model.solve()

print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

for var in model.variables():
  print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
  print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 1800.0
x1: 0.0
x2: 0.0
x3: 45.0
x4: 0.0
y1: 0.0
y3: 1.0
manpower: -5.0
material_a: -55.0
material_b: 0.0
x1_constraint: 0.0
x3_constraint: -55.0
y_constraint: 0.0
